# Building and analyzing EMMAA Models of Neurofibromatosis

As explained in the [README](http://github.com/indralab/nf_model), there are several steps that are used in the assembly of EMMAA models that currently require direct SQL access to the [INDRA Database](https://db.indra.bio). In particular, initializing and updating EMMAA models requires access to user-submitted curations and raw, unpreprocessed statements from the machine reading systems, neither of which are currently accessible via the public REST API of the INDRA Database (accessible at https://db.indra.bio). During the NF Hackathon we didn't have sufficient time to update EMMAA to run off strictly public APIs but we plan to do so in the near future.

With that in mind, in this notebook we show how we initialize the models for deploying to EMMAA, and we illustrate what EMMAA does on the platform to assemble them and generate causal explanations. These are generally the same steps and functions that are run in the AWS Batch jobs generated by the EMMAA system, but instead of reading/writing results to AWS S3 for visualization in the EMMAA web application we create and explore the results in this notebook.

This notebook illustrates the three key parts of our project:
1. Assembling an NF Knowledge Network from text mining results stored in the INDRA Database
2. Assembling a model of Ras-NF mechanisms from natural language descriptions of mechanisms
3. Explaining the causal relations in the text-mined model using the curated model
4. Using both models to explain NF drug-screening results

## 1. A Knowledge Network of Molecular Mechanisms from the Neurofibromatosis Literature

In this section we put together an initial neurofibromatosis-specific knowledge network based on molecular interactions we have compiled from reading all available abstracts and full text articles from PubMed.

As background, we use multiple text mining systems from our collaborators and from the research community to extract raw causal relations and then assemble them using our knowledge assembly system, INDRA (the Integrated Network and Dynamical Reasoning Assembler). We do this systematically across all available literature and store the results in the INDRA Database for use in downstream projects like this one. The INDRA Database is publicly accessible at https://db.indra.bio. 

The first step in creating the EMMAA model for Neurofibromatosis is to find publications broadly relevant for neurofibromatosis. We do this by querying based on the both the term "neurofibromatosis" as well as the top-level Medical Subject Heading (MESH) term for Neurofibromatoses, D017253 (see https://www.ncbi.nlm.nih.gov/mesh/68017253). We use the `indra.literature` module to search PubMed using the PubMed REST API.

In [46]:
# Preliminaries
import json
from mech_model.processor import NlModelProcessor
from indra.tools import assemble_corpus as ac
from indra.assemblers.pysb import PysbAssembler
from indra.explanation.model_checker.pysb import PysbModelChecker
from indra.literature import pubmed_client
from indra.pipeline import AssemblyPipeline

In [50]:
from emmaa.model import filter_eidos_ungrounded

In [51]:
# Query PubMed for articles related to NF
def get_nf_pmids():
    # PMIDs from text search for "neurofibromatosis"
    nf_text_pmids = pubmed_client.get_ids('neurofibromatosis')
    # Mesh term for neurofibromatoses
    nf_mesh_ids = pubmed_client.get_ids_for_mesh('D017253')
    return list(set(nf_text_pmids + nf_mesh_ids))

# Get the PMIDs
nf_pmids = get_nf_pmids()

Here's how many unique PMIDs we found:

In [52]:
len(nf_pmids)

17890

We then query the INDRA Database to get the INDRA Statements that have been extracted from these papers by text mining. One complication here is that for the EMMAA model we need to get raw, unassembled statements from the INDRA Database, which are currently not available from the public APIs (the public APIs provide access to the "finished", assembled statements). Therefore for this example we show the code that we run internally to obtain these statements but, by default, load the statements from a cache.

### Alternative 1: How we initialize the model for assembly on EMMAA using direct INDRA DB Access

In [6]:
# Get raw statement IDs from the DB for the given TextRefs
reload = False
pmid_stmts_file = 'lit_model/nf_raw_stmts.pkl'
if reload: 
    db = get_primary_db()
    # Get statements for the given text refs
    print(f"Distilling statements for {len(nf_pmids)} TextRefs")
    start = time.time()
    clauses = [
        db.TextRef.pmid.in_(nf_pmids),
        db.TextContent.text_ref_id == db.TextRef.id,
        db.Reading.text_content_id == db.TextContent.id,
        db.RawStatements.reading_id == db.Reading.id]
    pmid_stmts = distill_stmts(db, get_full_stmts=True, clauses=clauses)
    end = time.time()
    elapsed = end - start
    print(elapsed)
    list(ac.dump_statements(pmid_stmts, pmid_stmts_file))
else:
    pmid_stmts = list(ac.load_statements(pmid_stmts_file))

INFO: [2020-11-13 14:35:17] indra.tools.assemble_corpus - Loading lit_model/nf_raw_stmts.pkl...
INFO: [2020-11-13 14:35:17] indra.tools.assemble_corpus - Loaded 20209 statements


Let's see how many statements we got:

In [7]:
len(pmid_stmts)

20209

In [8]:
# Alternative placeholder--query INDRA DB Rest API for statements
# The next step is to get the INDRA Statements that have been extracted from these
#and stored in the INDRA Database, which will give us a sense of which molecular entities
#(proteins, drugs, etc.) are relevant to NF and should be included in the model.
if False:
    stmts = []
    batch_size = 100
    total_batches = int(len(nf_pmids) / batch_size) + 1
    for batch_ix, pmid_batch in enumerate(batch_iter(nf_pmids, 100)):
        print(batch_ix + 1, "of", total_batches)
        idrp = idr.get_statements_for_paper([('pmid', pmid) for pmid in pmid_batch])
        stmts.extend(idrp.statements)
    #for idrp = idr.get_statements_for_paper([('pmid', pmid) for pmid in nf_pmids][0:100])

### Preprocessing, filtering and deduplicating statements

The statements that we've obtained from the INDRA Database in the previous step represent raw output from NLP systems. We need to preprocess these statements in several ways: we need to fix named entity normalization, standardize sites of posttranslation modifications to canonical protein sequences, identify duplicate statements and overlapping statements, etc. The steps of this procedure are described in the `config.json` file for the EMMAA model. The EMMAA system reads the `config.json` file and runs assembly on the raw statements in AWS Batch. For illustration purposes we do this ourselves here.

In [40]:
# Load the config file
with open('lit_model/config.json', 'rt') as f:
    nf_config = json.load(f)

Here are the assembly steps for the model:
* `filter_no_hypothesis`. Filter out relations labeled by NLP systems as hypotheses (e.g. "we tested whether X causes Y")
* `ground_statements`. Runs entity normalization.
* `map_grounding`. Corrects common entity normalization errors and disambiguates entities using [Adeft](https://github.com/indralab/adeft).
* `filter_grounded_only`. Filters out any unnormalized entity texts. This can be removed to see if there are any useful entities that can't be linked to existing identifiers.
* `map_sequence`. Normalize sites of post-translational modification to canonical protein sequences. See ProtMapper [GitHub](https://github.com/indralab/protmapper) and [Preprint](https://www.biorxiv.org/content/10.1101/822668v3).
* `filter_eidos_ungrounded`. Filters out ungrounded entities in relations produced by the [Eidos](https://github.com/clulab/eidos) reader. 
* `run_preassembly`. Combine duplicate statements and organize statements by their relative specificity and detail.
* `filter_human_only`. Filter out references to non-human proteins.
* `filter_by_curation`. Filter out statements curated as incorrect. Note that because we don't have direct access to the INDRA Database to get these curations we skip this step that would run on EMMAA.

In [41]:
nf_config['assembly']

[{'function': 'filter_no_hypothesis'},
 {'function': 'ground_statements',
  'kwargs': {'mode': 'local', 'ungrounded_only': True}},
 {'function': 'ground_statements',
  'kwargs': {'mode': 'local',
   'sources': ['sparser'],
   'ungrounded_only': False}},
 {'function': 'map_grounding'},
 {'function': 'filter_grounded_only'},
 {'function': 'map_sequence'},
 {'function': 'filter_eidos_ungrounded'},
 {'function': 'run_preassembly', 'kwargs': {'return_toplevel': False}},
 {'function': 'filter_human_only'},
 {'function': 'filter_by_curation',
  'args': [{'function': 'get_curations'},
   'any',
   ['correct', 'act_vs_amt', 'hypothesis']],
  'kwargs': {'update_belief': True}}]

Remove the curation filter, as described above:

In [42]:
filter_by_curation_ix = 9
nf_config['assembly'].pop(filter_by_curation_ix)

{'function': 'filter_by_curation',
 'args': [{'function': 'get_curations'},
  'any',
  ['correct', 'act_vs_amt', 'hypothesis']],
 'kwargs': {'update_belief': True}}

Create the assembly pipeline and run:

In [43]:
ap = AssemblyPipeline(nf_config['assembly'])
pmid_stmts_asmb = ap.run(pmid_stmts)

In [44]:
pmid_stmts_asmb = ap.run(pmid_stmts)

INFO: [2020-11-13 15:30:13] indra.pipeline.pipeline - Running the pipeline
INFO: [2020-11-13 15:30:13] indra.pipeline.pipeline - Calling filter_no_hypothesis
INFO: [2020-11-13 15:30:13] indra.tools.assemble_corpus - Filtering 20209 statements to no hypothesis...
INFO: [2020-11-13 15:30:13] indra.tools.assemble_corpus - 19962 statements after filter...
INFO: [2020-11-13 15:30:13] indra.pipeline.pipeline - Calling ground_statements
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: ras p21
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing ras p21 with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: nf1 osx
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing Nf1 Osx with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: zinc finger protein
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing zinc finger protein with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Lo

INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing Neonatal airway obstruction with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: lats1/2fbd, lats1/2 fbd
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing Lats1/2-FBD with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: merlin ferm, merlinferm
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing Merlin-FERM with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: large tumor suppressor 2
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing large tumor suppressor 2 with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: raf/c
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing RAF/C with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: treatment of ch157mn and iomm lee cells with dovitinib, treatment of ch157mn and iommlee cell

INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing RASA1 variants with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: caπllary malformationarteriovenous malformation syndrome, capillary malformation arteriovenous malformation syndrome, capillary malformationarteriovenous malformation syndrome
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing capillary malformation-arteriovenous malformation syndrome with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: limk1/2
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing LIMK1/2 with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: mir 296 3p, mir2963p
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing miR-296-3p with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: egfr ras erk, egfrraserk
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing EGFR-Ras-ERK with 0 entrie

INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: many challenges in management
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing many challenges in management with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: apparent difference
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing apparent difference with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: hypothesize that in conjunction with yap
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing hypothesize that in conjunction with YAP with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: autosomal dominant genetic disorder
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing autosomal dominant genetic disorder with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: treatment options, treatment option
INFO: [2020-11-13 15:30:13] gilda.g

INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: potential bias, potential bia
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing potential bias with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: ce6
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing Ce6 with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: pl cddp ce6, plcddpce6
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing PL-cDDP-Ce6 with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: nmdar
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing NMDAR with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: other lesion, other lesions
INFO: [2020-11-13 15:30:13] gilda.grounder - Comparing other lesions with 0 entries
INFO: [2020-11-13 15:30:13] gilda.grounder - Looking up the following strings: profilin/g
INFO: [2020-11-13 15:30

...

INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: merlin
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing merlin with 3 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for merlin
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: 1433, 14 3 3
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing 14-3-3 with 2 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for 14-3-3
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: erk
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing ERK with 4 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for ERK
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: merlin
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing merlin with 3 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for merlin
INFO: [2020-11-13 15:30:51] gilda.g

INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: wt, wts
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing Wts with 3 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for Wts
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: hot
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing hot with 1 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for hot
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: pet
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing PET with 2 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for PET
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: pak1
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing Pak1 with 5 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for Pak1
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the f

INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing E3 with 4 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for E3
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: erm
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing ERM with 1 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for ERM
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: merlin
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing merlin with 3 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for merlin
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: akt
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing Akt with 3 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for Akt
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: p38
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing p38 wi

INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for Nf1
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: plp
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing Plp with 4 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for Plp
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: clamp
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing clamp with 1 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for clamp
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: ns
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing NS with 6 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation for NS
INFO: [2020-11-13 15:30:51] gilda.grounder - Looking up the following strings: iq
INFO: [2020-11-13 15:30:51] gilda.grounder - Comparing IQ with 2 entries
INFO: [2020-11-13 15:30:51] gilda.grounder - Running disambiguation

INFO: [2020-11-13 15:33:13] indra.preassembler - Finding refinements for 15 Autophosphorylation statements
INFO: [2020-11-13 15:33:13] indra.preassembler - Total comparisons: 4
INFO: [2020-11-13 15:33:13] indra.preassembler - Finding refinements for 1331 Complex statements
INFO: [2020-11-13 15:33:14] indra.preassembler - Total comparisons: 1388
INFO: [2020-11-13 15:33:14] indra.preassembler - Finding refinements for 4 Deacetylation statements
INFO: [2020-11-13 15:33:14] indra.preassembler - Total comparisons: 0
INFO: [2020-11-13 15:33:14] indra.preassembler - Finding refinements for 229 DecreaseAmount statements
INFO: [2020-11-13 15:33:14] indra.preassembler - Total comparisons: 51
INFO: [2020-11-13 15:33:14] indra.preassembler - Finding refinements for 4 Dehydroxylation statements
INFO: [2020-11-13 15:33:14] indra.preassembler - Total comparisons: 0
INFO: [2020-11-13 15:33:14] indra.preassembler - Finding refinements for 4 Demethylation statements
INFO: [2020-11-13 15:33:14] indra.pre

## 2. A Mechanistic Ras-NF Model from Natural Language

Next we show how a second, distinct Ras-NF mechanistic model can be automatically assembled from a set of manually curated sentences describing molecular interactions. We call this approach "natural language modeling" and it is described in detail in our paper:

Gyori B.M.\*, Bachman J.A.\*, Subramanian K., Muhlich J.L., Galescu L., Sorger P.K. From word models to executable models of signaling networks using automated assembly (2017), Molecular Systems Biology, 13, 954. (https://www.embopress.org/doi/full/10.15252/msb.20177651)


### Getting the statements from text

In this step we process the assertions in the text file `ras_nf_model.rst` into INDRA Statements. Each sentence is processed individually and the statements for each sentence are stored in a local cache. Processing can take several minutes.

In [9]:
model_text_file = 'ras_nf_model.rst'
stmt_output_file = 'ras_nf_stmts.pkl'

nlmp = NlModelProcessor(model_text_file, cache_dir='cache', trips_endpoint='drum-dev')
ras_stmts = nlmp.get_statements(stmt_output_file)

Processing group 'EGFR Pathway'...
1 of 13: processing 'EGF binds to EGFR.'...


INFO: [2020-11-11 16:15:12] processor - Saving 1 stmts in EGFbindstoEGFR.pkl


2 of 13: processing 'EGFR bound to EGF binds EGFR bound to EGF.'...


INFO: [2020-11-11 16:15:15] processor - Saving 1 stmts in EGFRboundtoEGFbindsEGFRboundtoEGF.pkl


3 of 13: processing 'EGFR bound to EGFR phosphorylates itself at Y1016.'...


...

INFO: [2020-11-11 16:21:40] processor - Saving 1 stmts in ActiveTIAM1activatesRAC1.pkl


Processing group 'Miscellaneous'...
1 of 2: processing 'Unphosphorylated PEBP1 binds RAF1.'...


INFO: [2020-11-11 16:21:41] processor - Saving 1 stmts in UnphosphorylatedPEBP1bindsRAF1.pkl


2 of 2: processing 'RAF1 not bound to PEBP1 is active.'...


INFO: [2020-11-11 16:21:43] processor - Saving 1 stmts in RAF1notboundtoPEBP1isactive.pkl
INFO: [2020-11-11 16:21:43] indra.tools.assemble_corpus - Dumping 210 statements into ras_nf_stmts.pkl...


Let's see how many statements we have:

In [10]:
len(ras_stmts)

210

The NLP system we're using for this, called TRIPS (developed by the Institute for Human and Machine Cognition, IHMC), sometimes grounds the text "NF1" as a cell line instead of as the gene/protein. So we fix that here using the INDRA `map_grounding` function.

In [15]:
# Grounding map the statements
grounding_map = {'NF1': {'HGNC': '7765'}}
ras_stmts = ac.map_grounding(ras_stmts, do_rename=True, grounding_map=grounding_map)

INFO: [2020-11-11 16:28:00] indra.preassembler.grounding_mapper.disambiguate - Adeft will not be available for grounding disambiguation.
INFO: [2020-11-11 16:28:00] indra.preassembler.grounding_mapper.disambiguate - INDRA DB is not available for text content retrieval for grounding disambiguation.
INFO: [2020-11-11 16:28:00] indra.tools.assemble_corpus - Mapping grounding on 210 statements...
INFO: [2020-11-11 16:28:00] indra.preassembler.grounding_mapper.disambiguate - INDRA DB is not available for text content retrieval for grounding disambiguation.
INFO: [2020-11-11 16:28:00] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/johnbachman/.indra/bio_ontology/1.4/bio_ontology.pkl
INFO: [2020-11-11 16:28:07] indra.preassembler.grounding_mapper.mapper - 0 statements filtered out


To deploy the model in EMMAA, we would then run the following script:

    `python scripts/emmaa_model_from_statements.py -

## Assembling an executable PySB model

To explain the claims that we extracted from the text mined NF model, we take our detailed natural language model and assemble it into a rule-based executable model using INDRA and PySB. For more on PySB see our previous paper:

Lopez C.F.\*, Muhlich J.L.\*, Bachman J.A.\*, Sorger P.K. Programming biological models in Python using PySB (2013), Molecular Systems Biology, 9, 646. (https://www.embopress.org/doi/full/10.1038/msb.2013.1)

In [19]:
ras_stmts = stmts

In [21]:
pa = PysbAssembler(ras_stmts)
pysb_model = pa.make_model()

INFO: [2020-11-11 16:46:54] indra.assemblers.pysb.assembler - Using default assembly policy.


Getting parents for ('HGNC', '20249')
Getting parents for ('HGNC', '7765')
Getting parents for ('HGNC', '11714')
Getting parents for ('HGNC', '16262')
Getting parents for ('HGNC', '8975')
Getting parents for ('HGNC', '6407')
Getting parents for ('HGNC', '8975')
Getting parents for ('HGNC', '6125')
Getting parents for ('HGNC', '8816')
Getting parents for ('HGNC', '8993')
Getting parents for ('HGNC', '12363')
Getting parents for ('HGNC', '12362')
Getting parents for ('HGNC', '12362')
Getting parents for ('HGNC', '12363')
Getting parents for ('HGNC', '3942')
Getting parents for ('HGNC', '28426')
Getting parents for ('HGNC', '3942')
Getting parents for ('HGNC', '10011')
Getting parents for ('HGNC', '6769')
Getting parents for ('HGNC', '6770')
Getting parents for ('HGNC', '9829')
Getting parents for ('HGNC', '8630')
Getting parents for ('HGNC', '3942')
Getting parents for ('HGNC', '30287')
Getting parents for ('HGNC', '3942')
Getting parents for ('HGNC', '30287')
Getting parents for ('HGNC'

## Checking the NF literature findings against our detailed model

Next we use the model checker component within INDRA to see how many of findings from our text mined NF model can be explained with the detailed mechanisms we've explicitly curated in our natural language model.

In [24]:
pmc = PysbModelChecker(pysb_model, nf_stmts[0:50])

NameError: name 'nf_stmts' is not defined

# New heading